In [1]:
import numpy as np
def BB(w0,wt,t,m):
    h = 2**m
    T = np.linspace(0,t, h+1)
    W = np.zeros(h+1)
    z = np.random.randn(h+1)
    W[0] = w0
    W[h] = wt
    j_max = 1
    
    for k in range(1, m+1):
        i_min = h//2
        i = i_min
        l = 0
        r = h
        for j in range(1, j_max+1):
            a = ((T[r]-T[i])*W[l] + (T[i]-T[l])*W[r]) / (T[r]-T[l])
            b = np.sqrt( (T[i]-T[l])*(T[r]-T[i])/(T[r]-T[l]) )
            W[i] = a + b*z[i]
            i+=h; l+=h; r+=h
        j_max*=2
        h = i_min
    
    return W

In [2]:
import numpy as np
def control_variable(r,q1,q2,sigma1,sigma2,rho,M,m):

    S1_start = 100
    S2_start = 100
    principal = 100    
    
    T = 2
    check_term = 0.5
    check_max = int(T / check_term)
    
    case=6
    S1 = np.zeros(check_max+1)
    S2 = np.zeros(check_max+1)
    Put1 = np.zeros(check_max)
    Put2 = np.zeros(check_max)
    digital = np.zeros(check_max)
    X = np.zeros([M, check_max*5])
    Y = np.zeros(M)
    prob = np.zeros(case)
    
    for i in range(M):
        
        S1[0] = S1_start
        S2[0] = S2_start
        
        x1 = np.random.randn(check_max)
        x2 = np.random.randn(check_max)
        W1 = np.zeros(check_max+1)
        W2 = np.zeros(check_max+1)

        check_barrier = False
        
        for j in range(1, check_max+1):

            # Random walk
            W1[j] = W1[j-1] + np.sqrt(check_term) * x1[j-1]
            W2[j] = W2[j-1] + np.sqrt(check_term) * x2[j-1]
            
            e1 =  W1[j] - W1[j-1]
            e2 = rho*e1 + (W2[j] - W2[j-1])*np.sqrt(1-rho**2)
    
            # Get S1, S2 price at early redemption time
            S1[j] = S1[j-1]*np.exp( (r - q1 - (sigma1**2)/2)*check_term + sigma1*e1)
            S2[j] = S2[j-1]*np.exp( (r - q2 - (sigma2**2)/2)*check_term + sigma2*e2)
            
            # Get Put1, Put2 price at early redemption time
            Put1[j-1] = max(S1_start * (0.85 -0.05*(j-1)) - S1[j] ,0) * np.exp(-r * j * check_term)
            Put2[j-1] = max(S2_start * (0.85 -0.05*(j-1)) - S2[j] ,0) * np.exp(-r * j * check_term)
            
            # Get digital option price at early redemption time
            if (S1[j] >= S1_start * (0.85 - ((j-1) * 0.05))) and (S2[j] >= S2_start * (0.85 - ((j-1) * 0.05))):
                digital[j-1] = 1 * np.exp(-r * j * check_term)
            else:
                digital[j-1] = 0
            
            X[i][j-1] = S1[j]
            X[i][j+3] = S2[j]
            X[i][j+7] = Put1[j-1]
            X[i][j+11] = Put2[j-1]
            X[i][j+15] = digital[j-1]
            
            
        for j in range(1, check_max+1):
            
            # Check Barrier
            if (S1[j] < S1_start * 0.6) or (S2[j] < S2_start * 0.6):
                check_barrier = True
            
            # Check early repayment
            if (S1[j] >= S1_start * (0.85 - ((j-1) * 0.05))) and (S2[j] >= S2_start * (0.85 - ((j-1) * 0.05))):
                Y[i] = principal * (1+0.0625*j) * np.exp(-r * j * check_term)
                prob[j-1]+=1
                break
            
            # Check Maturity repayment
            if j == check_max:
                if (check_barrier == True) and ( (S1[j] < S1_start * 0.7) or (S2[j] < S2_start * 0.7) ):
                    Y[i] = principal * min(S1[j]/S1_start, S2[j]/S2_start) * np.exp(-r * j * check_term)
                    prob[j+1]+=1
                    break
                    
                else:
                    S1_BB = np.zeros(2**m+1)
                    S2_BB = np.zeros(2**m+1)
                    S1_BB[0] = S1_start
                    S2_BB[0] = S2_start
                    
                    # Brownian Bridge
                    for k in range(1, check_max+1):      
                        W1_BB = BB(W1[k-1], W1[k], 0.5, m)
                        W2_BB = BB(W2[k-1], W2[k], 0.5, m)
                        
                        for l in range(1, 2**m+1):
                            e1 = W1_BB[l] - W1_BB[l-1]
                            e2 = rho*e1 + (W2_BB[l] - W2_BB[l-1]) * np.sqrt(1-rho**2)
                        
                            S1_BB[l] = S1_BB[l-1] * np.exp( (r - q1 - sigma1**2/2)*(check_term/2**m) + sigma1 * e1 )
                            S2_BB[l] = S2_BB[l-1] * np.exp( (r - q2 - sigma2**2/2)*(check_term/2**m) + sigma2 * e2 )
                        
                            if(S1_BB[l] < S1_start * 0.6) or (S2_BB[l] < S2_start * 0.6):
                                check_barrier = True
                                Y[i] = principal * min(S1[j]/S1_start, S2[j]/S2_start) * np.exp(-r * j * check_term)
                                prob[j+1]+=1
                                break
                                
                        if check_barrier:
                            break          
                        else:        
                            S1_BB[0] = S1_BB[2**m]
                            S2_BB[0] = S2_BB[2**m]
                    
                    if check_barrier == False:
                        Y[i] = principal * np.exp(-r * j * check_term)
                        prob[j]+=1
                        break
        
    return Y,X

r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=100000
m=3
Y,X=control_variable(r,q1,q2,sigma1,sigma2,rho,M,m)


In [3]:
np.cov(X,rowvar=False)

array([[ 4.79383836e+02,  4.84400696e+02,  4.93105547e+02,
         5.01268073e+02,  1.21929022e+02,  1.23291619e+02,
         1.27610133e+02,  1.27452569e+02, -7.06199053e+01,
        -6.78230355e+01, -5.96941634e+01, -5.06020456e+01,
        -2.55352026e+01, -2.40761341e+01, -2.12385891e+01,
        -1.76504109e+01,  4.96528702e+00,  3.53167739e+00,
         2.82256485e+00,  2.34733751e+00],
       [ 4.84400696e+02,  1.00163113e+03,  1.01992913e+03,
         1.03738385e+03,  1.22244489e+02,  2.50767690e+02,
         2.56367971e+02,  2.57047036e+02, -7.13338937e+01,
        -1.24757402e+02, -1.11023818e+02, -9.47084214e+01,
        -2.58889745e+01, -4.78952340e+01, -4.19289061e+01,
        -3.55775138e+01,  5.01116539e+00,  6.88716057e+00,
         5.48515745e+00,  4.61430756e+00],
       [ 4.93105547e+02,  1.01992913e+03,  1.58559377e+03,
         1.61114361e+03,  1.24541430e+02,  2.51934228e+02,
         3.97436651e+02,  4.00021269e+02, -7.25104189e+01,
        -1.26608813e+02, -1.5

In [4]:
import pandas as pd
x=np.cov(X,rowvar=False)
df=pd.DataFrame(x)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,479.383836,484.400696,493.105547,501.268073,121.929022,123.291619,127.610133,127.452569,-70.619905,-67.823036,-59.694163,-50.602046,-25.535203,-24.076134,-21.238589,-17.650411,4.965287,3.531677,2.822565,2.347338
1,484.400696,1001.631133,1019.929127,1037.383848,122.244489,250.767690,256.367971,257.047036,-71.333894,-124.757402,-111.023818,-94.708421,-25.888975,-47.895234,-41.928906,-35.577514,5.011165,6.887161,5.485157,4.614308
2,493.105547,1019.929127,1585.593768,1611.143610,124.541430,251.934228,397.436651,400.021269,-72.510419,-126.608813,-155.669200,-134.327810,-26.376552,-48.252589,-63.630111,-55.030473,5.073451,6.976279,8.096008,6.847568
3,501.268073,1037.383848,1611.143610,2233.923416,124.954657,255.833358,402.780990,543.505674,-73.813239,-128.579526,-158.167765,-169.376819,-26.633451,-49.522632,-65.239863,-74.329490,5.148959,7.097711,8.214648,8.933695
4,121.929022,122.244489,124.541430,124.954657,851.298843,863.480933,876.423168,886.842938,-20.335151,-18.323501,-15.673234,-13.203474,-152.420044,-147.089182,-132.668120,-114.867171,7.933972,5.683866,4.668340,4.002206
5,123.291619,250.767690,251.934228,255.833358,863.480933,1831.992876,1852.360446,1879.741503,-20.708035,-38.063472,-32.003019,-26.742189,-153.844025,-268.673390,-247.634347,-218.210833,8.046685,11.072087,9.147131,7.886831
6,127.610133,256.367971,397.436651,402.780990,876.423168,1852.360446,2936.838261,2983.743651,-21.076728,-38.168918,-47.820121,-39.567808,-155.542888,-272.689186,-344.738197,-307.988979,8.137875,11.195076,13.392981,11.590610
7,127.452569,257.047036,400.021269,543.505674,886.842938,1879.741503,2983.743651,4215.833373,-20.861482,-38.555236,-49.437338,-54.117276,-156.364505,-275.140145,-348.923188,-384.893932,8.184842,11.335874,13.579999,15.078807
8,-70.619905,-71.333894,-72.510419,-73.813239,-20.335151,-20.708035,-21.076728,-20.861482,28.601938,20.846514,16.709226,13.468931,5.262213,4.789547,4.188961,3.400437,-1.203066,-0.753379,-0.591959,-0.494478
9,-67.823036,-124.757402,-126.608813,-128.579526,-18.323501,-38.063472,-38.168918,-38.555236,20.846514,46.432646,34.870161,27.551290,4.452273,8.977799,7.711553,6.417733,-1.012195,-1.515864,-1.145840,-0.950344


In [5]:
a=np.column_stack((Y,X[:,15]))
R2=np.zeros(20)
for i in range(20):
    if i!=15:
        a3=np.column_stack((a,X[:,i]))
        c=np.cov(a3,rowvar=False)
        sxy=c[1:3,0]
        sxx=c[1:3,1:3]
        R2[i]=(np.matmul(sxy.reshape(1,-1),np.matmul(np.linalg.inv(sxx),sxy.reshape(-1,1)))/c[0,0])[0,0]
R2

array([0.4150304 , 0.41615822, 0.41072792, 0.40276588, 0.38756966,
       0.37755235, 0.37012411, 0.36657664, 0.44934049, 0.48952684,
       0.50218977, 0.4917119 , 0.42181662, 0.43149647, 0.40908584,
       0.        , 0.47491926, 0.45018788, 0.44056515, 0.4368677 ])

In [6]:
R2=np.zeros((20,20))
for i in range(20):
    a=np.column_stack((Y,X[:,i]))
    for j in range(20):
        if i!=j:
            a3=np.column_stack((a,X[:,j]))
            c=np.cov(a3,rowvar=False)
            sxy=c[1:3,0]
            sxx=c[1:3,1:3]
            R2[i,j]=(np.matmul(sxy.reshape(1,-1),np.matmul(np.linalg.inv(sxx),sxy.reshape(-1,1)))/c[0,0])[0,0]
R2

array([[0.        , 0.09009209, 0.09799926, 0.09946065, 0.16615941,
        0.1890967 , 0.18792545, 0.17916365, 0.11273405, 0.17060972,
        0.1988581 , 0.20081212, 0.24123188, 0.36371801, 0.41342492,
        0.4150304 , 0.2275775 , 0.25796149, 0.2987908 , 0.34960186],
       [0.09009209, 0.        , 0.09038537, 0.09349336, 0.18317006,
        0.18956688, 0.19029417, 0.18290415, 0.13182101, 0.1660562 ,
        0.19395729, 0.19625701, 0.25797655, 0.36274315, 0.41368503,
        0.41615822, 0.24431772, 0.25141769, 0.29243088, 0.34306508],
       [0.09799926, 0.09038537, 0.        , 0.08587402, 0.18779142,
        0.19589928, 0.18554559, 0.17892376, 0.14001699, 0.17437466,
        0.19042967, 0.19179326, 0.26257823, 0.37001159, 0.40819704,
        0.41072792, 0.25340225, 0.25971763, 0.28702813, 0.33710745],
       [0.09946065, 0.09349336, 0.08587402, 0.        , 0.18577728,
        0.19488573, 0.18527241, 0.17133129, 0.14126927, 0.17801658,
        0.19401933, 0.18783957, 0.26043494, 0

In [7]:
import pandas as pd
df=pd.DataFrame(R2)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000000,0.090092,0.097999,0.099461,0.166159,0.189097,0.187925,0.179164,0.112734,0.170610,0.198858,0.200812,0.241232,0.363718,0.413425,0.415030,0.227577,0.257961,0.298791,0.349602
1,0.090092,0.000000,0.090385,0.093493,0.183170,0.189567,0.190294,0.182904,0.131821,0.166056,0.193957,0.196257,0.257977,0.362743,0.413685,0.416158,0.244318,0.251418,0.292431,0.343065
2,0.097999,0.090385,0.000000,0.085874,0.187791,0.195899,0.185546,0.178924,0.140017,0.174375,0.190430,0.191793,0.262578,0.370012,0.408197,0.410728,0.253402,0.259718,0.287028,0.337107
3,0.099461,0.093493,0.085874,0.000000,0.185777,0.194886,0.185272,0.171331,0.141269,0.178017,0.194019,0.187840,0.260435,0.369619,0.408962,0.402766,0.255803,0.263323,0.290645,0.332939
4,0.166159,0.183170,0.187791,0.185777,0.000000,0.158285,0.168332,0.169606,0.203730,0.263158,0.290660,0.289963,0.207200,0.328748,0.380993,0.387570,0.236745,0.274947,0.317390,0.368217
5,0.189097,0.189567,0.195899,0.194886,0.158285,0.000000,0.154371,0.158866,0.226057,0.268349,0.297433,0.298187,0.239171,0.322492,0.372559,0.377552,0.268997,0.263745,0.306399,0.357111
6,0.187925,0.190294,0.185546,0.185272,0.168332,0.154371,0.000000,0.141480,0.224769,0.270187,0.287946,0.289513,0.250526,0.332184,0.368369,0.370124,0.280003,0.276746,0.294579,0.344847
7,0.179164,0.182904,0.178924,0.171331,0.169606,0.158866,0.141480,0.000000,0.215688,0.262956,0.281326,0.275013,0.252355,0.337987,0.372063,0.366577,0.281360,0.280874,0.300066,0.336435
8,0.112734,0.131821,0.140017,0.141269,0.203730,0.226057,0.224769,0.215688,0.000000,0.174941,0.209540,0.216729,0.273446,0.396948,0.447067,0.449340,0.239105,0.281733,0.325117,0.376577
9,0.170610,0.166056,0.174375,0.178017,0.263158,0.268349,0.270187,0.262956,0.174941,0.000000,0.205032,0.220192,0.334759,0.431655,0.484835,0.489527,0.298491,0.285082,0.335402,0.389206


In [8]:
np.max(R2)

0.5021897695080962

In [9]:
np.argmax(R2)

215

In [10]:
np.argmax(R2)//20

10

In [11]:
np.argmax(R2)%20

15

In [12]:
R2[10,15]

0.5021897695080962

In [13]:
import numpy as np
from scipy.stats import norm
def BS_put(S,K,r,q,T,sigma):
    d1=(np.log(S/K)+(r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2=d1-sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp(-q*T)*norm.cdf(-d1)
def ELS_multiple_control(r,q1,q2,sigma1,sigma2,rho,M,m):

    Y,X=control_variable(r,q1,q2,sigma1,sigma2,rho,M,m)
    
    k = np.where(Y >= 100 * 1.25 * np.exp(-r*2), 4, 
        np.where(Y >= 100 * 1.1875 * np.exp(-r*1.5), 3,
                np.where(Y >= 100 * 1.125 * np.exp(-r*1.0), 2,
                        np.where(Y >= 100 * 1.0625 * np.exp(-r*0.5), 1,
                                np.where(Y >= 100 * np.exp(-r*2), 5, 6)))))
    
    key, counts = np.unique(k, return_counts=True)
    prob = np.array(counts)
    
    value = np.mean(Y)
    err = np.std(Y) / np.sqrt(M)
    prob = prob / M
    
    mx = np.array( [np.mean(X[:,15]), np.mean(X[:,10])] )
    EX = np.array( [BS_put(100,70,r,q2,2,sigma2), BS_put(100,75,r,q1,1.5,sigma1)] )
    
    tmp_mtx=np.column_stack((Y,X[:,15],X[:,10]))
    tmp_cov=np.cov(tmp_mtx,rowvar=False)
    sxy=tmp_cov[1:3,0].reshape(-1,1)
    sxx=tmp_cov[1:3,1:3]
    
    b = np.linalg.inv(sxx).dot(sxy)
    R2 = (sxy.T).dot(b) / tmp_cov[0,0]
    
    value2 = ( value - b.T.dot( (mx - EX).reshape(-1,1) ) ).item()
    err2 = ( err * np.sqrt(1-R2) ).item()
    
    return value,value2,err,err2,prob,b,mx,EX
from datetime import datetime
r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=10000
m=3
t1=datetime.now()
value,value2,err,err2,prob,b,mx,EX=ELS_multiple_control(r,q1,q2,sigma1,sigma2,rho,M,m)
t2=datetime.now()
print('value = {:.2f}, value2 = {:.2f}, err = {:.2f}, err2 = {:.2f}'.format(value,value2,err,err2))
for i in range(6):
    print('prob[{:d}] = {:{width}.2%}'.format(i+1,prob[i],width=6))
print('Total sum of prob = {:.0%}'.format(sum(prob)))
print('Total computing time = {:f} seconds'.format((t2-t1).total_seconds()))

value = 92.52, value2 = 93.04, err = 0.27, err2 = 0.19
prob[1] = 54.69%
prob[2] = 11.42%
prob[3] =  6.62%
prob[4] =  4.43%
prob[5] =  0.33%
prob[6] = 22.51%
Total sum of prob = 100%
Total computing time = 0.731789 seconds


In [14]:
b

array([[-1.35501356],
       [-1.41662526]])

In [15]:
mx

array([6.26785367, 3.19858536])

In [16]:
EX

array([6.0256176, 3.0616534])

In [17]:
err2/err

0.6953311468738619

In [18]:
np.sqrt(1-R2[10,15])

0.7055566812750792